In [ ]:
# Лабораторная работа №6
## Выполнил студент группы БСТ2003 Григорян Арсен Григорович

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math
import time

In [ ]:
# Считывание графа
def read_graph(filename):
    adj_matrix = list()
    graphfile = open(filename, 'r')
    for l in graphfile:
        l = l.split()
        for i in range(len(l)):
            l[i] = int(l[i])
        adj_matrix.append(l)
    graphfile.close()
    return adj_matrix

# Отрисовка графа
def draw_graph(adj_matrix, path):
    graph = nx.DiGraph(np.matrix(adj_matrix))
    colors = []
    for item in graph.edges():
        catch = 0
        for i in range(len(path) - 1):
            if path[i] == item[0] and path[i + 1] == item[1]:
                catch += 1
                break
        if catch == 1:
            colors.append('red')
        else:
            colors.append('black')
    pos = nx.circular_layout(graph)
    labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw_networkx_nodes(graph, pos, node_size=250, node_color='royalblue')
    nx.draw_networkx_edges(graph, pos, edge_color=colors, arrows=True, arrowsize=10, width=2)
    nx.draw_networkx_labels(graph, pos, font_size=13, font_weight='bold', font_family='sans-serif')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, font_size=11, font_family='sans-serif')
    plt.axis('off')
    plt.show()

# Конвертация матрциы
def convert_adj_matrix(adj_matrix):
    res = []
    for i in range(len(adj_matrix)):
        res.append([])
        for j in range(len(adj_matrix)):
            res[i].append(adj_matrix[i][j])
            if adj_matrix[i][j] == 0 and i != j:
                res[i][j] = math.inf
    return res

# Алгоритм Левита
def levit(adj_matrix, start, end):
    adj_matrix = convert_adj_matrix(adj_matrix)
    M0 = set()
    M1 = {'main': [start], 'urgent': []}
    M2 = set(range(1, len(adj_matrix) + 1))
    dist = [math.inf for item in adj_matrix]
    dist[start] = 0
    paths = [[start] for item in adj_matrix]
    while len(M1['main']) or len(M1['urgent']):
        if len(M1['urgent']):
            u = M1['urgent'].pop(0)
        else:
            u = M1['main'].pop(0)
        for v in range(len(adj_matrix)):
            if dist[u] == math.inf or adj_matrix[u][v] == math.inf:
                continue
            if v in M2:
                M2.remove(v)
                M1['main'].append(v)
                if dist[v] > dist[u] + adj_matrix[u][v]:
                    paths[v] = paths[u].copy()
                    paths[v].append(v)
                    dist[v] = dist[u] + adj_matrix[u][v]
            elif v in M1['main'] or v in M1['urgent']:
                if dist[v] > dist[u] + adj_matrix[u][v]:
                    paths[v] = paths[u].copy()
                    paths[v].append(v)
                    dist[v] = dist[u] + adj_matrix[u][v]
            elif v in M0 and dist[v] > dist[u] + adj_matrix[u][v]:
                M0.remove(v)
                M1['urgent'].append(v)
                dist[v] = dist[u] + adj_matrix[u][v]
                paths[v] = paths[u].copy()
                paths[v].append(v)
        M0.add(u)
    if len(paths[end]) == 1:
        return [], 0
    return paths, dist

adj_matrix = read_graph('Матрица.txt')
start = int(input('Начальная вершина: '))
end = int(input('Конечная вершина: '))
start_time = time.time()
paths, dist = levit(adj_matrix, start, end)
if paths and dist:
    print(f"Длина пути: {dist[end]}")
    print(time.time() - start_time)
    draw_graph(adj_matrix, paths[end])
else:
    print(f"Путь из {start} в {end} не существует")
    draw_graph(adj_matrix, [])